In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature


import trax_util as trx

import importlib

import time as TimeUtils

import sys
# import modules in other directories
sys.path.append('/glade/work/juliob/PyRegridding/Regridder/')

import PlotUtil as Pu
import esmfRegrid as erg


importlib.reload( trx )


In [ ]:
Ret=6378.1

## Read in a track file

In [ ]:
##################
# Get track file
###################
power_wind=(1./6.)**(0.11)
TrakFile = trx.rcp85fname(sst='sst1') 
trk=trx.readtrx( TrakFile  , power_wind=power_wind )

##################
# Get BaseName
###################
BaseName = trx.rcp85fname(sst='sst1',justBaseName=True) 
print( f"BaseName is  {BaseName} " )
print( f"TrakFile is  {TrakFile} " )

nstorms,ntraxtime = np.shape( trk.lat )


In [ ]:
basename = BaseName + 'cam.h4.PRECT.'
basename_o = BaseName + 'cam.h4.TCPRECT.'
# input and output Directories
drc='/glade/campaign/cgd/ccr/nanr/HPSS/tokeep/TIMESLICE/atm/proc/tseries/hourly3/PRECT/'
drc_o='/glade/campaign/cgd/amp/juliob/TC-cesm1/precip/'

y=2071
yA=str(y).zfill(4)
fname=drc+basename+yA+'010100Z-'+yA+'123121Z.nc'
fname_o=drc_o+basename_o+yA+'010100Z-'+yA+'123121Z.nc'


In [ ]:
dU=xr.open_dataset( fname_o )
print(list(dU.variables))
nt,ny,nx = np.shape( dU.TCPRECT )
lon=dU.lon.values
lat=dU.lat.values

In [ ]:
poo=np.zeros((ny,nx)) 
for t in np.arange( nt ):
    poo=poo+dU.TCPRECT[t,:,:].values
    if ((t % 100)==0):
        print( nt, t )


In [ ]:
TCprecav=1000.*86400.*poo/nt

preclv=np.linspace(0.1,2.1,num=11) 
projPC = ccrs.Robinson(central_longitude=0.)
data_crs = ccrs.PlateCarree()
fig=plt.figure(figsize=(20,12))

ax1 = fig.add_subplot(1, 1, 1 , projection=projPC )

ax1.coastlines(resolution='110m',color='black',linewidth=4)
ax1.contourf( lon, lat, TCprecav ,levels=preclv,cmap='Blues',transform=data_crs )
for ns in np.arange( nstorms ):
    if (trk.year[ns,0]==2071):
        zoo=ax1.plot( trk.lon[ns,:], trk.lat[ns,:], transform=data_crs  )


ax1.spines['geo'].set_linewidth(4)  # change as needed

In [ ]:
#### STOP HERE???